In [ ]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 300
rcParams["figure.dpi"] = 300

In [ ]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import la_forge.core as co
import corner
from IPython.display import display, Math
from IPTA_DR2_analysis.model_blocks import lite_Tspan, full_Tspan

In [ ]:
import numpy as np
from scipy.special import erfc, erfcinv
import os, glob, pickle
import json
import matplotlib.pyplot as plt
import libstempo as t2
from enterprise.signals import utils as ent_utils
from enterprise import constants as const
from dr3_noise.models import model_singlepsr_noise
from enterprise_extensions.blocks import common_red_noise_block
from enterprise.signals import signal_base
from h5pulsar import Pulsar, FilePulsar
#from enterprise.pulsar import Pulsar
import IPTA_DR2_analysis.utils as dr2u
from IPTA_DR2_analysis.model_blocks import adv_noise_block

In [ ]:
from la_forge import core as co
import defiant
from defiant import OptimalStatistic
from defiant import utils, orf_functions
from defiant import plotting as defplot
from defiant.extra import mdc1_utils
import tqdm
from IPython.display import display, Math

In [ ]:
dataset = 'full'
gwb_Nfreqs = 13
crn_model = f'CRN{gwb_Nfreqs}_g4p3_advnoise'
# ------------
project_path = '/vast/palmer/home.grace/bbl29/IPTA_DR2_analysis'
datadir = f'{project_path}/data/{dataset}_ePSRs'
if dataset == 'edr2':
    core_save_dir = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/edr2'
    figsave = f'{project_path}/figs/edr2/{crn_model}'
else:
    core_save_dir = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/dr2{dataset}'
    figsave = f'{project_path}/figs/dr2{dataset}/{crn_model}'
if 'lite' in dataset:
    color = 'C1'
elif dataset == 'edr2':
    color = 'C2'
else:
    color = 'C0'

In [ ]:
chain_idxs = []
ln_weights = []
for path in glob.glob(f'{core_save_dir}/{crn_model}/HD_reweighting/*'):
    chain_idxs.append(np.loadtxt(f'{path}/chain_idxs.txt'))
    ln_weights.append(np.loadtxt(f'{path}/ln_weights.txt'))
chain_idxs = np.array(np.concatenate(chain_idxs), dtype=int)
ln_weights = np.concatenate(ln_weights)
chain_idxs = chain_idxs[~np.isnan(ln_weights)]
ln_weights = ln_weights[~np.isnan(ln_weights)]

In [ ]:
c = co.Core(corepath=f'{core_save_dir}/{crn_model}/core.h5', burn=0)

## Get Bayes fac

In [ ]:
Ns = len(ln_weights)
mean_wgt = np.nanmean(np.exp(ln_weights))
sigma_wgt = np.nanstd(np.exp(ln_weights))
neff = Ns/(1 + (sigma_wgt/mean_wgt)**2)
PLR = np.exp(ln_weights)#/np.sqrt(neff)
sigma_B = sigma_wgt/np.sqrt(neff)
print(Ns)

In [ ]:
display(Math(fr'N_{{\rm{{eff}}}} = {int(neff)}'))
display(Math(fr'\mathcal{{E}} = {neff/Ns:0.2f}'))
display(Math(fr'\mathcal{{B}}^{{\rm{{HD}}}}_{{\rm{{CRN}}}} = {mean_wgt:0.3f} \pm {sigma_B:0.3f}'))

In [ ]:
plt.figure(figsize=(7,3))
plt.hist(PLR, bins=40, histtype='step', density=True, range=(0,np.min([4,np.max(BF)])), label=r'$w$')
plt.axvline(1, alpha=0.5, color='k', label='BF = 1')
plt.axvline(10**0.3, color='C1',
            label=(r'Antoniadis+2022 result ($\mathcal{B}^{\rm{HD}}_{\rm{CARN}} =$'+
                   f'{np.round(10**0.3, decimals=3)})'))
plt.axvline(mean_wgt, color='k',
            label=r'My result ($\mathcal{B}^{\rm{HD}}_{\rm{CARN}} =$'+f'{np.round(mean_wgt, decimals=3)})')
plt.axvline(mean_wgt+sigma_B, color='k', ls='dashed')
plt.axvline(mean_wgt-sigma_B, color='k', ls='dashed')
plt.xlabel(r'$\mathcal{B}^{\rm{HD}}_{\rm{CARN}}$')
plt.ylabel('PDF')
plt.xlim([0,4])
plt.title('BF results from reweighting')
plt.legend()

## Fixed gamma results

In [ ]:
gwb_log10_A = c('crn_log10_A')[chain_idxs]

In [ ]:
fig, ax = plt.subplots(figsize=(3.5,2.2), dpi=500)
ax.hist(gwb_log10_A, histtype='step', density=True, bins=30, label='Auto correlations only')
ax.hist(gwb_log10_A, histtype='step', density=True, bins=30, label='Auto+cross correlations',
        weights=np.exp(ln_weights))
ax.set_xlabel(r'$\log_{10}A_{\rm{CRN}}$')
ax.set_ylabel('PDF')
ax.legend(fontsize='xx-small')
ax.set_title(f'DR2{dataset} | fixed gamma')
fig.tight_layout()
fig.savefig(f'{figsave}/hd_log10_A.png', dpi=500, bbox_inches='tight')

In [ ]:
# what does the CURN posterior look like over the samples where HD is favored? Over where it's not?

In [ ]:
len(PLR), len(chain_idxs)

In [ ]:
gwb_log10_A = c('crn_log10_A')[chain_idxs]
HD_log10_A = c('crn_log10_A')[chain_idxs][PLR > 1]
CRN_log10_A = c('crn_log10_A')[chain_idxs][PLR < 1]

In [ ]:
fig, ax = plt.subplots(figsize=(3.5,2.2), dpi=500)
ax.hist(gwb_log10_A, histtype='step', density=True, bins=30, label='Auto correlations only')
ax.hist(gwb_log10_A, histtype='step', density=True, bins=30, label='Auto+cross correlations',
        weights=np.exp(ln_weights))
ax.hist(HD_log10_A, histtype='step', density=True, bins=30, label='Samples where PLR > 1')
ax.hist(CRN_log10_A, histtype='step', density=True, bins=30, label='Samples where PLR < 1')
#ax.hist(gwb_log10_A, histtype='step', density=True, bins=30, label='Auto+cross correlations',
#        weights=np.exp(ln_weights))
ax.set_xlabel(r'$\log_{10}A_{\rm{CRN}}$')
ax.set_ylabel('PDF')
ax.legend(fontsize='xx-small')
ax.set_title(f'DR2{dataset} | fixed gamma')
fig.tight_layout()
#fig.savefig(f'{figsave}/hd_log10_A.png', dpi=500, bbox_inches='tight')

## Sanity check using the OS

The max vs min values of the distribution of weights should net HD plots that are visibly in preference for vs against the CURN

In [ ]:
# make CURN model
Nfreqs = 13
if 'lite_unfiltered' in dataset:
    noisedict_path = f'{project_path}/noisedicts/dr2lite_unfiltered_advnoise.json'
else:
    noisedict_path = f'{project_path}/noisedicts/dr2full_advnoise.json'
    
psrs = []
psr_paths = np.sort(glob.glob(f'{project_path}/data/{dataset}_ePSRs/J*'))
try:
    for psr_path in psr_paths:
        with open(psr_path,'rb') as f:
            psrs.append(FilePulsar(f))
except:
    for psr_path in psr_paths:
        with open(psr_path,'rb') as f:
            psrs.append(pickle.load(f))
len(psrs), psr_paths[0]

# load noise params
with open(noisedict_path,'r') as f:
    noise_params = json.load(f)

# set up model
max_mjd = np.max([np.max(psr.toas) for psr in psrs])
min_mjd = np.min([np.min(psr.toas) for psr in psrs])
Tspan = max_mjd - min_mjd
crn = common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan,
                             components=Nfreqs, gamma_val=13/3,
                             logmin=-18, logmax=-12,
                             orf=None, name='crn')

# loop through pulsars, add noise
models = []
for psr in tqdm.tqdm(psrs):
    noise = adv_noise_block(psr, full_pta_analysis=True, dataset='full', psr_model=True,
                            tm_marg=True, tm_svd=True)
    signals = crn + noise
    models.append(signals(psr))
pta = signal_base.PTA(models)
pta.set_default_params(noise_params)

In [ ]:
OS_obj = OptimalStatistic(psrs, pta=pta, gwb_name='crn', core=c, orfs=['hd'])
OS_obj.set_orf(['hd'])

In [ ]:
idx_best = np.argmax(PLR)
idx_worst = np.argmin(PLR)
params_best = {p:c(p)[chain_idxs][idx_best] for p in c.params}
params_worst = {p:c(p)[chain_idxs][idx_worst] for p in c.params}
print(PLR[idx_best], PLR[idx_worst])
print(params_best['crn_log10_A'], params_worst['crn_log10_A'])
print(params_best['lnlike'], params_worst['lnlike'])
print(params_best['J1909-3744_red_noise_log10_A'], params_worst['J1909-3744_red_noise_log10_A'])

In [ ]:
# If params=None, then DEFIANT will use maximum likelihood values from OS_obj.lfcore
xi,rho,sig,C,A2,A2s = OS_obj.compute_OS(N=1,params=params_best, gamma=13/3, pair_covariance=True)

print(f'{A2*1e30:0.2f} +- {A2s*1e30:0.2f} x 1e30')
print(f'{A2/A2s:0.2f}')

fig, ax = plt.subplots(figsize=(5,3))
xiavg, rhoavg, sigavg = defiant.utils.binned_pair_correlations(xi, rho, sig, bins=10, orf='hd')
ax.errorbar(xiavg, rhoavg, yerr=sigavg, marker='o', color='k', ls='none', capsize=5)
hd_orf = orf_functions.get_orf_function('hd')
xi_grid = np.linspace(0.0001,np.pi,1000)
ax.plot(xi_grid, A2*hd_orf(xi_grid), color=color)
ax.fill_between(xi_grid, (A2+A2s)*hd_orf(xi_grid), (A2-A2s)*hd_orf(xi_grid), color=color, alpha=0.2)
ax.set_xlim([0,np.pi])
ylim = ax.get_ylim()
ax.scatter(xi, rho, marker='.', s=1, color='k')
ax.set_ylim([-2.5e-29,2.5e-29])#2*np.array(ylim))
ax.set_title(f'{dataset} OS (Best sample)')
ax.set_xlabel('Pulsar separation (radians)')
ax.set_ylabel('Correlated power')
fig.savefig(f'{figsave}/OS_best_sample.png', dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
# If params=None, then DEFIANT will use maximum likelihood values from OS_obj.lfcore
xi,rho,sig,C,A2,A2s = OS_obj.compute_OS(N=1,params=params_worst, gamma=13/3, pair_covariance=True)

print(f'{A2*1e30:0.2f} +- {A2s*1e30:0.2f} x 1e30')
print(f'{A2/A2s:0.2f}')

fig, ax = plt.subplots(figsize=(5,3))
xiavg, rhoavg, sigavg = defiant.utils.binned_pair_correlations(xi, rho, sig, bins=10, orf='hd')
ax.errorbar(xiavg, rhoavg, yerr=sigavg, marker='o', color='k', ls='none', capsize=5)
hd_orf = orf_functions.get_orf_function('hd')
xi_grid = np.linspace(0.0001,np.pi,1000)
ax.plot(xi_grid, A2*hd_orf(xi_grid), color=color)
ax.fill_between(xi_grid, (A2+A2s)*hd_orf(xi_grid), (A2-A2s)*hd_orf(xi_grid), color=color, alpha=0.2)
ax.set_xlim([0,np.pi])
ylim = ax.get_ylim()
ax.scatter(xi, rho, marker='.', s=1, color='k')
ax.set_ylim([-2.5e-29,2.5e-29])#2*np.array(ylim))
ax.set_title(f'{dataset} OS (Worst sample)')
ax.set_xlabel('Pulsar separation (radians)')
ax.set_ylabel('Correlated power')
fig.savefig(f'{figsave}/OS_worst_sample.png', dpi=400, bbox_inches='tight')
plt.show()

## Varied gamma results

In [ ]:
axis_labels = [r'$\log_{10}A_{\rm{GWB}}$', r'$\gamma_{\rm{GWB}}$']
params = [p for p in c.params if 'crn' in p]
p_idxs = [c.params.index(p) for p in params]
fig = corner.corner(c.chain[chain_idxs][:,p_idxs], levels=(0.68,0.95), labels=axis_labels,
                    color='C1', plot_density=False, plot_datapoints=False,
                    no_fill_contours=False, hist_kwargs={'density':True},
                    label_kwargs={'fontsize':20}, bins=30, weights=np.exp(ln_weights))
lines = [mlines.Line2D([],[],color='C0',label='Auto correlations only')]
fig = corner.corner(c.chain[chain_idxs][:,p_idxs], levels=(0.68,0.95), fig=fig,
                    color='C0', plot_density=False, plot_datapoints=False,
                    no_fill_contours=False, hist_kwargs={'density':True},
                    bins=30)
lines += [mlines.Line2D([],[],color='C1',label='Auto+cross correlations')]
fig.legend(handles=lines, fontsize=10)
axes = np.array(fig.axes).reshape((2,2))
axes[0,0].axvline(13/3, color='C2')
axes[1,0].axvline(13/3, color='C2')
axes[1,0].axhline(np.log10(2.8e-15), color='C2')
axes[1,1].axvline(np.log10(2.8e-15), color='C2')
axes[1,1].set_title(f'DR2{dataset} | varied gamma')
axes[1,0].set_ylim([-16,-13])
axes[1,1].set_xlim([-16,-13])
axes[0,0].set_xlim([2,6.1])
axes[1,0].set_xlim([2,6.1])
fig.savefig(f'{figsave}/hd_corner.png', dpi=300, bbox_inches='tight')
plt.show()

## Free spec results

We can't make a weighed violin plot, but we can plot the free spec bin histograms

In [ ]:
params = [p for p in c.params if 'crn' in p]
fig, ax = plt.subplots(int(np.ceil(len(params)/4)), 4, figsize=(8,2*int(np.ceil(len(params)/4))))
ax = np.array(ax).flatten()
for i in range(len(params)):
    ax[i].hist(c(params[i])[chain_idxs], density=True, color='C0', histtype='step', bins=30, label='CRN')
    ax[i].hist(c(params[i])[chain_idxs], density=True, color='C1', histtype='step', bins=30,
               weights=np.exp(ln_weights), label='HD')
    ax[i].set_xlabel(fr'$\log_{{10}}\rho_{i}$')
ax[1].legend(fontsize='small')
for i in range(int(np.ceil(len(params)/4))*4 - len(params)):
    ax[int(np.ceil(len(params)/4))*4-i-1].set_visible(False)
fig.suptitle(f'DR2{dataset} | free spec')
fig.savefig(f'{figsave}/hd_free_spec.png', dpi=300, bbox_inches='tight')
fig.tight_layout()

## Check pulsar noise params

In [ ]:
def make_labels(params, psrname, add_psrname=True):
    labels = []
    if add_psrname:
        add = psrname+'\n'
    else:
        add = ''
    for p in params:
        if 'dm_gp_gamma' in p:
            labels.append(add+r'$\gamma_{\rm{DM}}$')
        elif 'dm_gp_log10_A' in p:
            labels.append(add+r'$\log_{10}A_{\rm{DM}}$')
        elif 'red_noise_gamma' in p:
            labels.append(add+r'$\gamma_{\rm{IRN}}$')
        elif 'red_noise_log10_A' in p:
            labels.append(add+r'$\log_{10}A_{\rm{IRN}}$')
        elif p == 'crn_log10_A':
            labels.append(r'$\log_{10}A_{\rm{CRN}}$')
        elif p == 'crn_gamma':
            labels.append(r'$\gamma_{\rm{CRN}}$')
        elif 'exp1_log10_Amp' in p:
            labels.append(add+r'$\log_{10}A_{\rm{exp}}$')
        elif 'exp1_log10_tau' in p:
            labels.append(add+r'$\log_{10}\tau_{\rm{exp}}$')
        elif 'exp1_t0' in p:
            labels.append(add+r'$t_{\rm{exp}}$')
        else:
            labels.append(p)
    return labels

In [ ]:
psrnames = np.unique([p.split('_')[0] for p in c.params if 'J' in p])
for psrname in psrnames:
    corepath = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/dr2full/stdnoise/{psrname}/core.h5'
    #c_psr = co.Core(corepath=corepath, burn=0)
    params = [p for p in c.params if psrname in p] + [p for p in c.params if 'crn' in p]
    p_idxs = [c.params.index(p) for p in params]
    labels = make_labels(params, psrname, add_psrname=False)
    fig = corner.corner(c.chain[chain_idxs][:,p_idxs], levels=(0.68,0.95), labels=labels,
                        color='C0', plot_density=False, plot_datapoints=False,
                        no_fill_contours=True, hist_kwargs={'density':True},
                        label_kwargs={'fontsize':20})
    fig = corner.corner(c.chain[chain_idxs][:,p_idxs], levels=(0.68,0.95), fig=fig,
                        color='C1', plot_density=False, plot_datapoints=False, weights=np.exp(ln_weights),
                        no_fill_contours=True, hist_kwargs={'density':True})
    lines = [mlines.Line2D([],[],color='C0',label='CRN')]
    lines += [mlines.Line2D([],[],color='C1',label='HD (reweighting)')]
    fig.legend(handles=lines, fontsize=20)
    fig.suptitle(f'PSR {psrname}', fontsize=30)
    plt.show()
    '''# next add single pulsar run
    p_psr_idxs = [c_psr.params.index(p) for p in params]
    fig = corner.corner(c_psr.chain[:,p_psr_idxs], levels=(0.68,0.95), fig=fig,
                        color='C0', plot_density=False, plot_datapoints=False,
                        no_fill_contours=True, hist_kwargs={'density':True})
    # finally add CRN onto intrinsic red noise plot
    ndim = len(params)
    axes = np.reshape(fig.axes, (ndim,ndim))
    idx_crn = c.params.index('crn_log10_A')
    axes[-1,-1].hist(c.chain[:,idx_crn], color='C1',
                     density=True, bins=20, histtype='step')
    # add 13/3 line
    axes[-1,-2].axvline(13/3, color='C1', ls='dashed')
    axes[-2,-2].axvline(13/3, color='C1', ls='dashed')
    lines = [mlines.Line2D([],[],color='C0',label='Single run')]
    lines += [mlines.Line2D([],[],color='k',label='PTA run')]
    lines += [mlines.Line2D([],[],color='C1',label='CRN')]
    fig.legend(handles=lines, fontsize=20)
    fig.suptitle(f'PSR {psrname}', fontsize=30)
    fig.savefig(f'{figsave}/{psrname}.png', dpi=300, bbox_inches='tight')'''

In [ ]:
plt.close('all')

In [ ]:
# KL divergence
np.log(np.mean(np.exp(ln_weights))) - np.mean(ln_weights)